In [1]:
import numpy as np
import pandas as pd
from func import *

np.set_printoptions(precision=3, suppress=True)

In [2]:
inf = np.finfo(np.float32).max
inf = 1_000_000

In [3]:
C = np.array([
    [ 0, 0],
    [10, 0],
    [ 4, 3],
    [10, 3],
    [12, 3],
])
assert np.shape(C)[1] == 2, "Las coordenadas se componen de 2 valores."

In [4]:
S = np.array([
    [0,0,0],
    [0,0,1],
    [1,1,1],
    [1,1,1],
    [1,1,1]
])
#assert np.shape(S)[1] == np.shape(C)[1], "La matriz S y la matriz C deben tener la misma forma."

In [5]:
B = np.array([
    [0,2],
    [2,3],
    [1,3],
    [3,4]
])

In [6]:
EAI = np.array([
    [1, inf, 1],
    [1, inf, 1],
    [1, inf, 1],
    [1, inf, 1]
])

In [7]:
L = np.linalg.norm(np.diff(C[B], axis=1), axis=2)
L

array([[5.],
       [6.],
       [3.],
       [2.]])

In [8]:
with np.errstate(divide='ignore'):
    alpha = np.arctan(np.divide(*(np.diff(C[B], axis=1).squeeze().T[::-1])))
np.rad2deg(alpha)

array([36.87,  0.  , 90.  ,  0.  ])

In [9]:
EAIL = np.hstack([EAI, L])

In [10]:
with np.errstate(divide='ignore', ):    
    Ki = []
    A = []
    Kt = []
    for el, t in zip(EAIL, alpha):
        Ki.append(stiffness_2d(*el))
        A.append(rot_2d(t))
        Kt.append(A[-1].T @ Ki[-1] @ A[-1])
    Ki = np.array(Ki)
    A = np.array(A)
    Kt = np.array(Kt)

In [11]:
def format(value) -> str:
    #if np.abs(value) > 10000:
    #    return 'inf'
    return str(round(value, 3))

pd.options.display.float_format = format

In [12]:
Kt.shape

(4, 6, 6)

In [13]:
show = pd.DataFrame(Kt[0])
show

,0,1,2,3,4,5
0,128000.035,95999.954,-0.144,-128000.035,-95999.954,-0.144
1,95999.954,72000.061,0.192,-95999.954,-72000.061,0.192
2,-0.144,0.192,0.8,0.144,-0.192,0.4
3,-128000.035,-95999.954,0.144,128000.035,95999.954,0.144
4,-95999.954,-72000.061,-0.192,95999.954,72000.061,-0.192
5,-0.144,0.192,0.4,0.144,-0.192,0.8


In [14]:
Kg = np.zeros(shape=(15,15))
GLg = np.zeros(shape=(15, 1))

In [15]:
_idx = np.arange(0, S.size)
G_idx = _idx.reshape(len(S), -1)[B].reshape(-1, 6)
for idx, Ki, GL in zip(G_idx, Kt, S[B]):
    Kg[*np.meshgrid(idx, idx)] += Ki
    # GLg[idx] += GL.ravel()
GLg = S.ravel()
Kg.shape, GLg.shape

((15, 15), (15,))

In [16]:
pd.DataFrame(Kg, index=_idx + 1, columns=_idx + 1)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,128000.035,95999.954,-0.144,0.0,0.0,0.0,-128000.035,-95999.954,-0.144,0.0,0.0,0.0,0.0,0.0,0.0
2,95999.954,72000.061,0.192,0.0,0.0,0.0,-95999.954,-72000.061,0.192,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.144,0.192,0.8,0.0,0.0,0.0,0.144,-0.192,0.4,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.444,0.0,-0.667,0.0,0.0,0.0,-0.444,-0.0,-0.667,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,333333.333,0.0,0.0,0.0,0.0,-0.0,-333333.333,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,-0.667,0.0,1.333,0.0,0.0,0.0,0.667,-0.0,0.667,0.0,0.0,0.0
7,-128000.035,-95999.954,0.144,0.0,0.0,0.0,294666.701,95999.954,0.144,-166666.667,0.0,0.0,0.0,0.0,0.0
8,-95999.954,-72000.061,-0.192,0.0,0.0,0.0,95999.954,72000.117,-0.025,0.0,-0.056,0.167,0.0,0.0,0.0
9,-0.144,0.192,0.4,0.0,0.0,0.0,0.144,-0.025,1.467,0.0,-0.167,0.333,0.0,0.0,0.0
10,0.0,0.0,0.0,-0.444,-0.0,0.667,-166666.667,0.0,0.0,666667.111,0.0,0.667,-500000.0,0.0,0.0


In [17]:
    pd.DataFrame(np.diag(GLg) @ Kg @ np.diag(GLg), index=_idx + 1, columns=_idx + 1)\
    .pipe(lambda x: x.loc[(x != 0).any(axis=0), (x != 0).any(axis=1)])

,6,7,8,9,10,11,12,13,14,15
6,1.333,0.0,0.0,0.0,0.667,-0.0,0.667,0.0,0.0,0.0
7,0.0,294666.701,95999.954,0.144,-166666.667,0.0,0.0,0.0,0.0,0.0
8,0.0,95999.954,72000.117,-0.025,0.0,-0.056,0.167,0.0,0.0,0.0
9,0.0,0.144,-0.025,1.467,0.0,-0.167,0.333,0.0,0.0,0.0
10,0.667,-166666.667,0.0,0.0,666667.111,0.0,0.667,-500000.0,0.0,0.0
11,-0.0,0.0,-0.056,-0.167,0.0,333334.889,1.333,0.0,-1.5,1.5
12,0.667,0.0,0.167,0.333,0.667,1.333,4.0,0.0,-1.5,1.0
13,0.0,0.0,0.0,0.0,-500000.0,0.0,0.0,500000.0,0.0,0.0
14,0.0,0.0,0.0,0.0,0.0,-1.5,-1.5,0.0,1.5,-1.5
15,0.0,0.0,0.0,0.0,0.0,1.5,1.0,0.0,-1.5,2.0


In [18]:
(np.multiply(*np.meshgrid(GLg, GLg)) != 0).shape

(15, 15)

In [19]:
Kg[np.multiply(*np.meshgrid(GLg, GLg)) != 0].reshape(10,10)

array([[      1.333,       0.   ,       0.   ,       0.   ,       0.667,
             -0.   ,       0.667,       0.   ,       0.   ,       0.   ],
       [      0.   ,  294666.701,   95999.954,       0.144, -166666.667,
              0.   ,       0.   ,       0.   ,       0.   ,       0.   ],
       [      0.   ,   95999.954,   72000.117,      -0.025,       0.   ,
             -0.056,       0.167,       0.   ,       0.   ,       0.   ],
       [      0.   ,       0.144,      -0.025,       1.467,       0.   ,
             -0.167,       0.333,       0.   ,       0.   ,       0.   ],
       [      0.667, -166666.667,       0.   ,       0.   ,  666667.111,
              0.   ,       0.667, -500000.   ,       0.   ,       0.   ],
       [     -0.   ,       0.   ,      -0.056,      -0.167,       0.   ,
         333334.889,       1.333,       0.   ,      -1.5  ,       1.5  ],
       [      0.667,       0.   ,       0.167,       0.333,       0.667,
              1.333,       4.   ,       0.   